In [42]:
# Requests allows you to send HTTP/1.1 requests extremely easily.
# There’s no need to manually add query strings to your URLs, or to form-encode your POST data. 
# Keep-alive and HTTP connection pooling are 100% automatic, thanks to urllib3.
import requests
import json
import pandas as pd
import csv
from pandas import json_normalize 
from operator import itemgetter
from user_key import userkey # Pulls in P1s Crunchbase API user key

In [43]:
def url_count(query, query_type): 
    # POST method with API URL, query_type as a parameter, and passing query as json.
    # https://www.w3schools.com/python/ref_requests_post.asp
    r = requests.post("https://api.crunchbase.com/api/v4/searches/" + query_type, params = userkey , json = query)
    #print(r.text)
    #print(r.url)
    result = json.loads(r.text)
    count = result["count"]
    return count

def url_extraction(query, query_type):    
    # Create global raw variable
    global raw   
    # POST method with API URL, query_type as a parameter, and passing query as json.
    # https://www.w3schools.com/python/ref_requests_post.asp
    r = requests.post("https://api.crunchbase.com/api/v4/searches/" + query_type, params = userkey , json = query)
    result = json.loads(r.text)
    normalized_raw = json_normalize(result['entities'])
    # Append normalized entity results to global raw variable
    raw = raw.append(normalized_raw, ignore_index=True)

def autocompletes(query, collection_ids_list=None, limit=None):
    '''
    Suggests matching Identifier entities based on the query and entity_def_ids provided.
    
    QUERY
    Value to perform the autocomplete search with.
    
    COLLECTION_IDS_LIST
    A comma separated list of collection ids to search against. 
    Leaving this blank means it will search across all identifiers. 
    Entity defs can be constrained to specific facets by providing them as facet collections. 
    Relationship collections will resolve to their underlying entity def.
    Collection ids are: organizations, people, funding_rounds, acquisitions, investments,
    events, press_references, funds, event_appearances, ipos, ownerships, categories, 
    category_groups, locations, jobs
    
    LIMIT
    Number of results to retrieve; default = 10, max = 25
    '''
    params = {**userkey, "query": query}
    if collection_ids_list and type(collection_ids_list) == list:
        params.update({"collection_ids": collection_ids_list})
    if limit and type(limit) == int:
        params.update({"limit": limit})
    r = requests.get("https://api.crunchbase.com/api/v4/autocompletes", params = params)
    result = json.loads(r.text)
    normalized_result = json_normalize(result["entities"])
    dataframe = pd.DataFrame.from_dict(normalized_result)
    return dataframe

def makequery_jobs_board_of_org(uuid_list, limit=1000):
    '''
    Job Search: Current Board Affiliations
    - Organization includes list of `uuid` values
    - Excludes `employee` and `executive` level jobs
    '''
    query = {
        "field_ids": [
            "entity_def_id",
            "identifier",
            "job_type",
            "name",
            "organization_identifier",
            "person_identifier",
            "short_description",
            "is_current",
            "started_on",
            "ended_on",
            "title",
            "updated_at",
            "uuid"],
        "limit": limit,
        "query": [
            {
                "type": "predicate",
                "field_id": "organization_identifier",
                "operator_id": "includes",
                "values": uuid_list
            },
            {
                "type": "predicate",
                "field_id": "job_type",
                "operator_id": "not_includes",
                "values": ["employee", "executive"]
            }]
    }
    return query

def people_entity(entity_id, field_ids_list=['primary_job_title', 'primary_organization', 'linkedin'], card_ids_list=None):
    '''
    
    ENTITY_ID
    UUID or permalink of desired entity
    
    FIELD_IDS
    Fields to include on the resulting entity - 
    either an array of field_id strings in JSON 
    or a comma-separated list encoded as string
    
    CARD_IDS
    Cards to include on the resulting entity - 
    array of card_id strings in JSON encoded as string\ 
    Card Ids for Person: [degrees, event_appearances, fields, 
    founded_organizations, jobs, participated_funding_rounds, 
    participated_funds, participated_investments, partner_funding_rounds, 
    partner_investments, press_references, primary_job, primary_organization]
    
    '''
    params = {**userkey}
    if field_ids_list and type(field_ids_list) == list:
        string = ""
        for item in field_ids_list:
            string += item+","
        string=string[:-1]
        params.update({"field_ids": string})
    if card_ids_list and type(card_ids_list) == list:
        string = ""
        for item in card_ids_list:
            string += item+","
        string=string[:-1]
        params.update({"card_ids": string})
    r = requests.get("https://api.crunchbase.com/api/v4/entities/people/"+str(entity_id), params = params)
    if r.text == "Usage limit exceeded":
        print('Usage Limited Exceeded, restart loop')
        return
    print(r.url)
    print(r.text)
    result = json.loads(r.text)
    uuid = result["properties"]["identifier"]["uuid"]
    try:
        linkedin = result["properties"]["linkedin"]["value"]
    except:
        linkedin = "NA"
    try:
        title = result["properties"]["primary_job_title"]
    except:
        title = "NA"
    try:
        org = result["properties"]["primary_organization"]["value"]
    except:
        org = "NA"
    
    return {uuid:title},{uuid:org},{uuid:linkedin}

# Pledge 1 Equity Company Board Members

## Pull UUIDs for list of companies, using `autocompletes` helper function

In [44]:
companies = pd.read_csv("crunchbase-api/Boardroom Allies Tracker - company_search_terms.csv").Company.to_list()
companies.sort()
uuid_companies = []
counter = 0
for company in companies:
    counter += 1
    print('*'*50)
    print("{}) Searching for {}".format(counter, company.upper()))
    found = autocompletes(company, ["organizations"], limit=1)
    uuid_companies.append(found["identifier.uuid"][0])
    print('*'*50)
    print("Found {} !!!!!!!\nDESCRIPTION: {}\n".format(found["identifier.value"][0].upper(), found["short_description"][0]))

**************************************************
1) Searching for APPFIRE
**************************************************
Found APPFIRE !!!!!!!
DESCRIPTION: Appfire is a provider of Atlassian Enterprise products and services.

**************************************************
2) Searching for ATLASSIAN
**************************************************
Found ATLASSIAN !!!!!!!
DESCRIPTION: Atlassian provides collaboration, development, and issue tracking software for teams.

**************************************************
3) Searching for AUTH0
**************************************************
Found AUTH0 !!!!!!!
DESCRIPTION: Auth0 operates a cloud-based identity platform for developers.

**************************************************
4) Searching for CANVA
**************************************************
Found CANVA !!!!!!!
DESCRIPTION: Canva is an online design and publishing platform that provides user friendly design tools for non-designers.

************************

**************************************************
Found ZUORA !!!!!!!
DESCRIPTION: Zuora provides cloud-based software on a subscription basis that helps companies launch, manage, and transform into a subscription business.



## Pull all current/former board affiliations for companies, using `makequery_jobs_board_of_org` helper function

In [45]:
query = makequery_jobs_board_of_org(uuid_companies) # Helper function
raw = pd.DataFrame() # Global raw variable
comp_count = url_count(query, "jobs") 
url_extraction(query, "jobs")
board_affiliations = raw[["properties.organization_identifier.value",
                    "properties.person_identifier.uuid",
                    "properties.person_identifier.value", 
                    "properties.title", 
                    "properties.job_type", 
                    "properties.is_current",
                    "properties.started_on.value",
                    "properties.ended_on.value",
                    "properties.updated_at"]].sort_values(["properties.organization_identifier.value"])
# Get UUIDs of people
uuid_board_members = list(set(board_affiliations["properties.person_identifier.uuid"].to_list()))
# Display
board_affiliations.head()

,properties.organization_identifier.value,properties.person_identifier.uuid,properties.person_identifier.value,properties.title,properties.job_type,properties.is_current,properties.started_on.value,properties.ended_on.value,properties.updated_at
330,Atlassian,6522f077-190d-e974-0743-81000498e74f,Ryan Sweeney,Director,board_member,False,2015-01-01,2017-01-01,2019-09-06T11:03:02Z
46,Atlassian,9da0aa4b-6fd1-9d12-cfb6-5073f7c86d9a,Steve Sordello,Director,board_member,True,2015-11-01,NaN,2019-09-06T13:26:35Z
307,Atlassian,08e82d6b-51a2-e642-23bf-05e52cbd9278,Murray Demo,Director,board_member,False,2011-12-01,2015-10-01,2019-09-06T13:05:36Z
206,Atlassian,822b9c18-cb8d-e494-68f3-6cb138620080,Kirk Bowman,Director,board_member,False,2012-01-01,2016-01-01,2019-09-06T12:19:37Z
112,Atlassian,d4e2a745-53ff-36e1-9960-17270d5b7e8c,Richard Wong,Director,board_member,True,2010-07-01,NaN,2019-09-06T13:05:36Z


In [46]:
# Test output
testerson = uuid_board_members[0]
people_entity(testerson)

https://api.crunchbase.com/api/v4/entities/people/6addd93e-f07c-1291-b244-7df3f582c4ff?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"linkedin":{"value":"https://www.linkedin.com/in/joshsteinvc/"},"identifier":{"uuid":"6addd93e-f07c-1291-b244-7df3f582c4ff","value":"Josh Stein","image_id":"zyo3rodyenor1jc5wqib","permalink":"josh-stein","entity_def_id":"person"}}}


({'6addd93e-f07c-1291-b244-7df3f582c4ff': 'NA'},
 {'6addd93e-f07c-1291-b244-7df3f582c4ff': 'NA'},
 {'6addd93e-f07c-1291-b244-7df3f582c4ff': 'https://www.linkedin.com/in/joshsteinvc/'})

## Find primary organization info, using `people_entity` helper function

In [47]:
# Start with empty dictionnaries
dict_title = {}
dict_org = {}
dict_linkedin = {}
no_primary_org = []

# For each API call, only update dictionary if it's not empty
for i,person in enumerate(uuid_board_members):
    print(i)
    try:
        dict_title_person, dict_org_person, dict_li_person = people_entity(person)
    if dict_title_person[person] != "NA":
        dict_title.update(dict_title_person)
    if dict_org_person[person] != "NA":
        dict_org.update(dict_org_person)
    if dict_li_person[person] != "NA":
        dict_linkedin.update(dict_li_person)
    if dict_title_person[person] == "NA" or dict_org_person[person] == "NA" or dict_li_person[person] == "NA":
        no_primary_org.append(person)

0
https://api.crunchbase.com/api/v4/entities/people/6addd93e-f07c-1291-b244-7df3f582c4ff?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"linkedin":{"value":"https://www.linkedin.com/in/joshsteinvc/"},"identifier":{"uuid":"6addd93e-f07c-1291-b244-7df3f582c4ff","value":"Josh Stein","image_id":"zyo3rodyenor1jc5wqib","permalink":"josh-stein","entity_def_id":"person"}}}
1
https://api.crunchbase.com/api/v4/entities/people/bd8d58c6-b287-c01d-3bc7-260ccc7a4b0b?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Co-Founder","primary_organization":{"uuid":"943de4f1-3b1d-4b09-a131-a48abf22535d","value":"Coalition","image_id":"di5rv3vvcbhpmygduoeg","permalink":"coalition-535d","entity_def_id":"organization"},"linkedin":{"value":"http://www.linkedin.com/in/johnhering"},"identifier":{"uuid":"bd8d58c6-b287-c01d-3bc7-260ccc7a4b0b","value":"John

https://api.crunchbase.com/api/v4/entities/people/fe73e532-a1e3-7bbf-cc69-5df480427e51?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Board member","primary_organization":{"uuid":"d6616327-34bc-abf8-2e22-f44fa064adaa","value":"ShopGo","image_id":"cgoawgfrpwilty5t4hth","permalink":"shopgo","entity_def_id":"organization"},"linkedin":{"value":"http://www.linkedin.com/in/davemcclure"},"identifier":{"uuid":"fe73e532-a1e3-7bbf-cc69-5df480427e51","value":"Dave McClure","image_id":"fi7bfp6l7yubkba7xdj1","permalink":"dave-mcclure","entity_def_id":"person"}}}
15
https://api.crunchbase.com/api/v4/entities/people/cda0aff1-eb50-7130-d775-c81f00ab448d?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"linkedin":{"value":"http://www.linkedin.com/in/lilatretikov"},"identifier":{"uuid":"cda0aff1-eb50-7130-d775-c81f00ab448d","value":"Lila Treti

https://api.crunchbase.com/api/v4/entities/people/d01eb97d-69e1-6213-bb0e-42b9268ba35b?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Board Advisor","primary_organization":{"uuid":"bf83106d-58d5-7c95-1dd4-7ccedc7ca017","value":"Auth0","image_id":"geps97mgxzao8eglnpsv","permalink":"auth0","entity_def_id":"organization"},"identifier":{"uuid":"d01eb97d-69e1-6213-bb0e-42b9268ba35b","value":"Tim Bray","image_id":"v1397186111/1a62b64c1743677ec12babeea04ce022.jpg","permalink":"tim-bray","entity_def_id":"person"}}}
28
https://api.crunchbase.com/api/v4/entities/people/270dc460-437d-50fb-3b5a-e2c9260bf26c?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"President","primary_organization":{"uuid":"f80fd5ed-a795-d90b-22e1-17b82ae2d321","value":"Southern New Hampshire University","image_id":"zxaibdoyk451wobrsdqy","perm

https://api.crunchbase.com/api/v4/entities/people/dc65bb64-eb46-42e9-26b7-ba215a39b049?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Founder and CEO","primary_organization":{"uuid":"25853aa5-b114-860b-2c2e-3377350e24da","value":"GrowthHackers","image_id":"v1475278439/p2hdb86tifbifucsfq4i.jpg","permalink":"growthhackers","entity_def_id":"organization"},"linkedin":{"value":"http://www.linkedin.com/in/seanellis"},"identifier":{"uuid":"dc65bb64-eb46-42e9-26b7-ba215a39b049","value":"Sean Ellis","image_id":"v1448352464/scnpftpruk8gsumppqem.png","permalink":"sean-ellis","entity_def_id":"person"}}}
41
https://api.crunchbase.com/api/v4/entities/people/431bfd77-1b58-819c-56c0-fdb84f65c9d5?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"General Partner","primary_organization":{"uuid":"1f4b4e0e-61b8-5e30-bf1e-bf8ef

https://api.crunchbase.com/api/v4/entities/people/f29d2c06-5ae9-a9ce-57ea-cb91df0d6e5e?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Member of the Board of Directors","primary_organization":{"uuid":"31a4e4b9-096e-12e5-1da4-d76f400bfcc0","value":"E*TRADE","image_id":"v1476500866/ukg833gylj7adlryeuig.png","permalink":"e-trade","entity_def_id":"organization"},"linkedin":{"value":"http://www.linkedin.com/in/bsaeger"},"identifier":{"uuid":"f29d2c06-5ae9-a9ce-57ea-cb91df0d6e5e","value":"Becky Saeger","image_id":"v1397184773/c546a82119c1bd062dfdb1accb10c503.jpg","permalink":"becky-saeger","entity_def_id":"person"}}}
53
https://api.crunchbase.com/api/v4/entities/people/fa68f08b-4f98-4c92-4601-57049fc6e30e?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Managing Partner","primary_organization":{"uuid":"8f34e608-

...

https://api.crunchbase.com/api/v4/entities/people/8299add3-e708-031d-b558-9374265611d6?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"CEO","primary_organization":{"uuid":"42758359-ba2b-e50d-80ef-21b4be9f2439","value":"Zapproved","image_id":"v1422906521/t6ti2ktw3qwamrk5but1.png","permalink":"zapproved","entity_def_id":"organization"},"linkedin":{"value":"http://www.linkedin.com/in/monicaenand"},"identifier":{"uuid":"8299add3-e708-031d-b558-9374265611d6","value":"Monica Enand","image_id":"mm6kx2mzxxpxtzzobeva","permalink":"monica-enand","entity_def_id":"person"}}}
237
https://api.crunchbase.com/api/v4/entities/people/b3547a83-54a7-0b6b-ccc1-726e2e7dbf68?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"CEO","primary_organization":{"uuid":"143764d8-9c9b-701f-7350-00b85f6cd2a7","value":"PagerDuty","image_id":"

https://api.crunchbase.com/api/v4/entities/people/46d201a1-d4b0-09ab-c0a2-cad5433245be?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Co-founder & Co-CEO","primary_organization":{"uuid":"7cb65346-6b0f-b133-ed27-21fcb25104ad","value":"Atlassian","image_id":"dvcq6aphn9unb1ypolhz","permalink":"atlassian","entity_def_id":"organization"},"linkedin":{"value":"https://au.linkedin.com/in/mcannonbrookes"},"identifier":{"uuid":"46d201a1-d4b0-09ab-c0a2-cad5433245be","value":"Mike Cannon-Brookes","image_id":"j0saebu3uy73nyqzudha","permalink":"mike-cannon-brookes","entity_def_id":"person"}}}
250
https://api.crunchbase.com/api/v4/entities/people/50c18c6b-4f67-3314-2b96-b2c9605cf49b?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"linkedin":{"value":"http://www.linkedin.com/in/mtanne"},"identifier":{"uuid":"50c18c6b-4f67-3314-2b96-b2c9605c

https://api.crunchbase.com/api/v4/entities/people/405abaa4-7f6f-41b5-98c4-e82a9dfab09b?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Founder & CEO","primary_organization":{"uuid":"456d8b38-6df8-4dc9-99a4-8d44904ce89f","value":"Birmingham Tech (Birmingham Tech Week)","image_id":"pvjoeb1sqcaoedtzugmq","permalink":"birmingham-tech-birmingham-tech-week","entity_def_id":"organization"},"linkedin":{"value":"https://www.linkedin.com/in/yiannismaos/"},"identifier":{"uuid":"405abaa4-7f6f-41b5-98c4-e82a9dfab09b","value":"Yiannis Maos","image_id":"mk0tyaapfikrsu5ejios","permalink":"yiannis-maos","entity_def_id":"person"}}}
264
https://api.crunchbase.com/api/v4/entities/people/d916a710-4c7f-1527-8461-dace686748ef?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"CEO","primary_organization":{"uuid":"758a79aa-cf07-1c15

https://api.crunchbase.com/api/v4/entities/people/7be70cb2-321f-d67e-70cc-bdba931d9a00?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"General Partner","primary_organization":{"uuid":"cc964bc0-a950-4d8f-4ef0-7c514e49d28a","value":"Emergence","image_id":"xeuj4qcnu0kfl6xs2dcs","permalink":"emergence-capital-partners","entity_def_id":"organization"},"linkedin":{"value":"http://www.linkedin.com/in/kevinspain"},"identifier":{"uuid":"7be70cb2-321f-d67e-70cc-bdba931d9a00","value":"Kevin Spain","image_id":"v1435899879/kgqtkvakxvx5jbxffeba.png","permalink":"kevin-spain","entity_def_id":"person"}}}
276
https://api.crunchbase.com/api/v4/entities/people/674e6dea-b01b-7a47-f88b-88bead81d5c3?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Co-founder & CEO","primary_organization":{"uuid":"cf6d3339-99d3-6ff2-4709-3f071d1

TypeError: cannot unpack non-iterable NoneType object

## Repeat if stopped by Crunchbase, otherwise ignore

In [48]:
for i,person in enumerate(uuid_board_members[280:]):
    print(i)
    dict_title_person, dict_org_person, dict_li_person = people_entity(person)
    if dict_title_person[person] != "NA":
        dict_title.update(dict_title_person)
    if dict_org_person[person] != "NA":
        dict_org.update(dict_org_person)
    if dict_li_person[person] != "NA":
        dict_linkedin.update(dict_li_person)
    if dict_title_person[person] == "NA" or dict_org_person[person] == "NA" or dict_li_person[person] == "NA":
        no_primary_org.append(person)

0
https://api.crunchbase.com/api/v4/entities/people/f9c96c90-5bb4-6ba0-bd38-04823b1d5b08?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Co-Founder & CEO","primary_organization":{"uuid":"6a2859a6-e096-926a-ee81-871ccbd16da8","value":"Slack","image_id":"hbnsvigizeltqp16sz9u","permalink":"slack","entity_def_id":"organization"},"linkedin":{"value":"http://www.linkedin.com/in/butterfield"},"identifier":{"uuid":"f9c96c90-5bb4-6ba0-bd38-04823b1d5b08","value":"Stewart Butterfield","image_id":"v1484796074/vczggrfvzkex8pwqwaxa.png","permalink":"stewart-butterfield","entity_def_id":"person"}}}
1
https://api.crunchbase.com/api/v4/entities/people/0a984955-bd12-3202-966e-7a789e862a3a?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"linkedin":{"value":"https://www.linkedin.com/in/brianokelley"},"identifier":{"uuid":"0a984955-bd12-3202-966e

https://api.crunchbase.com/api/v4/entities/people/b1880c1e-5b98-6fc7-b392-b4fbd070e744?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Global CEO","primary_organization":{"uuid":"f5d24aa6-f92e-96bf-7c9b-2966b120e5b0","value":"Opportunity International","image_id":"v1455368878/ewir4rfea9lpaqqhp0fj.png","permalink":"opportunity-international","entity_def_id":"organization"},"linkedin":{"value":"https://www.linkedin.com/in/vicki-escarra-a072b327"},"identifier":{"uuid":"b1880c1e-5b98-6fc7-b392-b4fbd070e744","value":"Vicki Escarra","image_id":"v1464274414/l7sjrvlvc3xftygnjwtg.png","permalink":"vicki-escarra","entity_def_id":"person"}}}
14
https://api.crunchbase.com/api/v4/entities/people/aade2215-0e2a-36a4-14d8-2b379da9e6b7?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"linkedin":{"value":"http://www.linkedin.com/in/daveschappel

https://api.crunchbase.com/api/v4/entities/people/6770ef58-fe52-2000-8c06-ee6708a4c6a8?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Executive Vice President and Chief Sales and Marketing Officer","primary_organization":{"uuid":"e0906c05-fae5-9591-ba5f-2142d8b0065a","value":"Cisco","image_id":"w0jp4bmv89nqrd83keix","permalink":"cisco","entity_def_id":"organization"},"linkedin":{"value":"https://www.linkedin.com/in/gerrielliott"},"identifier":{"uuid":"6770ef58-fe52-2000-8c06-ee6708a4c6a8","value":"Gerri Elliott","image_id":"v1397181326/7ae8c957095470f39da84ed65cae3ec1.jpg","permalink":"gerri-elliott","entity_def_id":"person"}}}
26
https://api.crunchbase.com/api/v4/entities/people/3853f5bd-d0ef-d7db-d1fc-6e584ff2a8d3?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Chief Executive Officer","primary_organiz

https://api.crunchbase.com/api/v4/entities/people/af6ea807-54ad-5a6b-21a2-b804b9a38a56?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Founder & Partner","primary_organization":{"uuid":"581c3a95-25e3-6c0c-4b5e-6e303e8df605","value":"Francisco Partners","image_id":"v1397178812/f6019725095b2a84f8fe929ae4a9f619.png","permalink":"francisco-partners","entity_def_id":"organization"},"linkedin":{"value":"https://www.linkedin.com/in/sandy-robertson-7220926"},"identifier":{"uuid":"af6ea807-54ad-5a6b-21a2-b804b9a38a56","value":"Sandy Robertson","image_id":"v1465099254/chhtpeudrgefwlbz9jq7.jpg","permalink":"sandy-robertson-2","entity_def_id":"person"}}}
39
https://api.crunchbase.com/api/v4/entities/people/8ac69662-00c3-9d15-35ac-cb7e032ff452?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Founder & Managing Director

https://api.crunchbase.com/api/v4/entities/people/c85b28d1-4a1a-6c12-a59b-78377a3811b8?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Partner and Managing Director","primary_organization":{"uuid":"a34c63f0-c277-3e4b-bbfc-1868ca81596d","value":"Menlo Ventures","image_id":"v1475585357/zrgz4eq7n6idd8ehycyf.jpg","permalink":"menlo-ventures","entity_def_id":"organization"},"linkedin":{"value":"http://www.linkedin.com/pub/venky-ganesan/0/3/544"},"identifier":{"uuid":"c85b28d1-4a1a-6c12-a59b-78377a3811b8","value":"Venky Ganesan","image_id":"v1461874320/yvkn9egadx67nvwtxzav.jpg","permalink":"venky-ganesan","entity_def_id":"person"}}}
52
https://api.crunchbase.com/api/v4/entities/people/d2f0fb7c-eb4c-2f47-3d13-497a07856796?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"General Partner","primary_organization":{"u

...

https://api.crunchbase.com/api/v4/entities/people/0f9f3c05-cb79-f58f-6cc5-98ddc8382d4f?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Co-Founder & MD","primary_organization":{"uuid":"8d5c7e48-82da-3025-dd46-346a31bab86f","value":"Meritech Capital Partners","image_id":"pgksugfwud8ivb5e6sag","permalink":"meritech-capital-partners","entity_def_id":"organization"},"linkedin":{"value":"http://www.linkedin.com/pub/paul-madera/1/a41/246"},"identifier":{"uuid":"0f9f3c05-cb79-f58f-6cc5-98ddc8382d4f","value":"Paul Madera","image_id":"hhl4gjwtaknxq4xm2ggc","permalink":"paul-madera","entity_def_id":"person"}}}
77
https://api.crunchbase.com/api/v4/entities/people/81cda5cf-0f4f-a6da-b62d-cb7f611a1dff?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Managing Director","primary_organization":{"uuid":"34a9bb66-0984-8de0-e

https://api.crunchbase.com/api/v4/entities/people/7609809e-b770-7057-c958-e7a4de8aad8b?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Member of the Advisory Board","primary_organization":{"uuid":"a21fd3fa-c9c5-be84-f26c-e1d78939ae6c","value":"DocuSign","image_id":"k13zxtz947k6nodkofvc","permalink":"docusign","entity_def_id":"organization"},"linkedin":{"value":"https://www.linkedin.com/in/richard-smith-80a7a618"},"identifier":{"uuid":"7609809e-b770-7057-c958-e7a4de8aad8b","value":"Richard Smith","image_id":"v1462731645/rxmst8wp3an8og5jfrvp.jpg","permalink":"richard-f-smith","entity_def_id":"person"}}}
90
https://api.crunchbase.com/api/v4/entities/people/c964b73b-40de-fa1c-2692-d391ca9aaeb4?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"CTO & Co-Founder","primary_organization":{"uuid":"143764d8-9c9b-701f-

https://api.crunchbase.com/api/v4/entities/people/a9f21aca-2ed7-a3f4-639c-9b4472a6cb24?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Managing Partner","primary_organization":{"uuid":"0333ae41-4131-4865-afc2-4199f3c357ec","value":"Bond","image_id":"iedckaglq9b4ysiwylfi","permalink":"bond-57ec","entity_def_id":"organization"},"linkedin":{"value":"https://www.linkedin.com/in/mary-meeker-5823ba48/"},"identifier":{"uuid":"a9f21aca-2ed7-a3f4-639c-9b4472a6cb24","value":"Mary Meeker","image_id":"v1410592403/la8u2nvqjhdkowqqvmc4.png","permalink":"mary-meeker","entity_def_id":"person"}}}
102
https://api.crunchbase.com/api/v4/entities/people/8692cd06-7e0b-f82c-1f32-087bf144cfd6?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"General Partner","primary_organization":{"uuid":"14474ae7-b6a1-a0a8-e2ea-f117cd1deed7","va

https://api.crunchbase.com/api/v4/entities/people/0dcbea3e-30e7-3776-885a-57588122c2de?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Founder","primary_organization":{"uuid":"2cd0606c-c371-84ec-0fac-cc2ecb423a6c","value":"Vlocity","image_id":"uvkxt4u7s1grhqhw5kgo","permalink":"vlocity","entity_def_id":"organization"},"linkedin":{"value":"https://www.linkedin.com/in/young-sohn-3a0a2992/"},"identifier":{"uuid":"0dcbea3e-30e7-3776-885a-57588122c2de","value":"Young Sohn","image_id":"gk4axiznxrn0rzsjkqtq","permalink":"young-sohn-3","entity_def_id":"person"}}}
115
https://api.crunchbase.com/api/v4/entities/people/e69099cb-7dd2-9077-27e9-021cf2eaee14?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Co-Founder & Managing Partner","primary_organization":{"uuid":"e8db05ed-6b48-9b54-e746-2f140dcc94be","value":"QED I

https://api.crunchbase.com/api/v4/entities/people/68ff8d03-5251-c102-99a2-91cbafc91f86?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"CTO","primary_organization":{"uuid":"7b9aff32-8364-68db-5c59-840c64a21d81","value":"2015 Special Olympics World Games","image_id":"v1451123294/ofdj1nwkmkgnipmnminc.png","permalink":"special-olympics","entity_def_id":"organization"},"linkedin":{"value":"https://www.linkedin.com/in/noahbroadwater"},"identifier":{"uuid":"68ff8d03-5251-c102-99a2-91cbafc91f86","value":"Noah Broadwater","image_id":"v1464273837/ce7guwnx3axpwzwui02w.png","permalink":"noah-broadwater","entity_def_id":"person"}}}
128
https://api.crunchbase.com/api/v4/entities/people/6c729290-27c0-3823-d122-4d0da6d08485?user_key=b3a00f4816ce8e8ce6657d0bf67baecf&field_ids=primary_job_title%2Cprimary_organization%2Clinkedin
{"properties":{"primary_job_title":"Board of Directors","primary_organization":{"uuid"

## Add person's primary title, organization, and LinkedIn to `board_affiliations` dataframe.

In [49]:
board_affiliations['Primary_Job_Title'] = board_affiliations['properties.person_identifier.uuid'].map(dict_title)
board_affiliations['Primary_Organization'] = board_affiliations['properties.person_identifier.uuid'].map(dict_org)
board_affiliations['LinkedIn'] = board_affiliations['properties.person_identifier.uuid'].map(dict_linkedin)
# Create CSV file 
board_affiliations.to_csv("OUT_board_affiliations.csv")
# Title, Org, LinkedIn added
board_affiliations.head()

,properties.organization_identifier.value,properties.person_identifier.uuid,properties.person_identifier.value,properties.title,properties.job_type,properties.is_current,properties.started_on.value,properties.ended_on.value,properties.updated_at,Primary_Job_Title,Primary_Organization,LinkedIn
330,Atlassian,6522f077-190d-e974-0743-81000498e74f,Ryan Sweeney,Director,board_member,False,2015-01-01,2017-01-01,2019-09-06T11:03:02Z,General Partner,Accel,http://www.linkedin.com/in/ryanjsweeney
46,Atlassian,9da0aa4b-6fd1-9d12-cfb6-5073f7c86d9a,Steve Sordello,Director,board_member,True,2015-11-01,NaN,2019-09-06T13:26:35Z,Chief Financial Officer,LinkedIn,https://www.linkedin.com/in/stevesordello
307,Atlassian,08e82d6b-51a2-e642-23bf-05e52cbd9278,Murray Demo,Director,board_member,False,2011-12-01,2015-10-01,2019-09-06T13:05:36Z,Member of the Board of Directors,Citrix Systems,http://www.linkedin.com/pub/murray-demo/17/311...
206,Atlassian,822b9c18-cb8d-e494-68f3-6cb138620080,Kirk Bowman,Director,board_member,False,2012-01-01,2016-01-01,2019-09-06T12:19:37Z,Venture Partner,Accel,http://www.linkedin.com/pub/kirk-bowman/0/2b/947
112,Atlassian,d4e2a745-53ff-36e1-9960-17270d5b7e8c,Richard Wong,Director,board_member,True,2010-07-01,NaN,2019-09-06T13:05:36Z,General Partner,Accel,http://www.linkedin.com/in/richpwong


In [50]:
# Count of how many are missing Title, Organization, or LinkedIn
len(no_primary_org)

78

In [51]:
# For each company, pull current board members
companies = list(set(board_affiliations["properties.organization_identifier.value"].to_list()))
companies.sort()

In [52]:
board_members_dict = {}
for org in companies:
    board_members_df = board_affiliations[(board_affiliations["properties.organization_identifier.value"]==org) &
                                          (board_affiliations["properties.is_current"]) &
                                          (board_affiliations["properties.job_type"]=="board_member")]
    board_members_names = board_members_df["properties.person_identifier.value"].to_list()
    board_members_org = board_members_df["Primary_Organization"].to_list()
    
    # Turn into string
    board_string = ""
    if board_members_names != []:
        board_info = dict(zip(board_members_names, board_members_org))
        # Add them to string
        for name, company in sorted(board_info.items()):
            if pd.isna(company):
                board_string += name + "; "
            else:
                board_string += name + " (" +company+"); "
        # Remove trailing comma
        board_string = board_string[:-2]
    board_members_dict[org] = board_string
with open('OUT_str_board_members_former.csv', 'w') as f:
    for key in board_members_dict.keys():
        f.write("%s, %s\n" % (key, board_members_dict[key]))

In [53]:
board_members_dict = {}
for org in companies:
    board_members_df = board_affiliations[(board_affiliations["properties.organization_identifier.value"]==org) &
                                          (board_affiliations["properties.is_current"] == False) &
                                          (board_affiliations["properties.job_type"]=="board_member")]
    board_members_names = board_members_df["properties.person_identifier.value"].to_list()
    board_members_org = board_members_df["Primary_Organization"].to_list()
    
    board_string = ""
    if board_members_names != []:
        board_info = dict(zip(board_members_names,board_members_org))
        # Add them to string
        for name, company in sorted(board_info.items()):
            if pd.isna(company):
                board_string += name + "; "
            else:
                board_string += name + " (" +company+"); "
        # Remove trailing comma
        board_string = board_string[:-2]
    board_members_dict[org] = board_string
with open('OUT_str_board_members_former.csv', 'w') as f:
    for key in board_members_dict.keys():
        f.write("%s, %s\n" % (key, board_members_dict[key]))

In [54]:
# For each company, pull current board affiliations
board_members_dict = {}
for org in companies:
    board_members_df = board_affiliations[(board_affiliations["properties.organization_identifier.value"]==org) &
                                          (board_affiliations["properties.is_current"]) &
                                          (board_affiliations["properties.job_type"]!="board_member")]
    board_members_names = board_members_df["properties.person_identifier.value"].to_list()
    board_members_org = board_members_df["Primary_Organization"].to_list()
    
    board_string = ""
    if board_members_names != []:
        board_info = dict(zip(board_members_names,board_members_org))
        # Add them to string
        for name, company in sorted(board_info.items()):
            if pd.isna(company):
                board_string += name + "; "
            else:
                board_string += name + " (" +company+"); "
        # Remove trailing comma
        board_string = board_string[:-2]
    board_members_dict[org] = board_string
import csv
with open('OUT_str_board_affiliations_current.csv', 'w') as f:
    for key in board_members_dict.keys():
        f.write("%s, %s\n" % (key, board_members_dict[key]))

In [55]:
# For each company, pull former board affiliations
board_members_dict = {}
for org in companies:
    board_members_df = board_affiliations[(board_affiliations["properties.organization_identifier.value"]==org) &
                                          (board_affiliations["properties.is_current"] == False) &
                                          (board_affiliations["properties.job_type"]!="board_member")]
    board_members_names = board_members_df["properties.person_identifier.value"].to_list()
    board_members_org = board_members_df["Primary_Organization"].to_list()
    
    board_string = ""
    if board_members_names != []:
        board_info = dict(zip(board_members_names,board_members_org))
        # Add them to string
        for name, company in sorted(board_info.items()):
            if pd.isna(company):
                board_string += name + "; "
            else:
                board_string += name + " (" +company+"); "
        # Remove trailing comma
        board_string = board_string[:-2]
    board_members_dict[org] = board_string
import csv
with open('OUT_str_board_affiliations_former.csv', 'w') as f:
    for key in board_members_dict.keys():
        f.write("%s, %s\n" % (key, board_members_dict[key]))